In [1]:
import pandas as pd
from pathlib import Path
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
raw_data_dir = Path.cwd() / "data" / "raw"
clean_data_dir = Path.cwd() / "data" / "clean"

# Taylor Rule

In [3]:
time_span = [year for year in range(1980, 2003)]

### Interest Rates

(OECD)

In [4]:
df = pd.read_csv(
    raw_data_dir / "ST_interestrate_1980-2002.csv",
    sep=";",  
)

In [5]:
df = df.rename(columns={"Year": "year", "Interest Rate": "interest_rate"})

In [6]:
df = df.loc[df["year"].isin(time_span)]

In [7]:
df["interest_rate"] = df["interest_rate"].apply(lambda x: x.replace(",", ".")).astype(float)

In [8]:
df.to_csv(clean_data_dir / "interest_rate.csv", sep="\t", index=False)

### Inflation

(WWW.RIVALUTA.it)

In [9]:
df = pd.read_csv(
    raw_data_dir / "datiinflazionemediaitalia.csv", 
    sep=";",  
)

In [10]:
df = df[1:-1]

In [11]:
df = df.rename(columns={"WWW.RIVALUTA.it": "inflation_rate"})

In [12]:
df.reset_index(col_level=0, names="year", inplace=True)

In [13]:
df = df.sort_values("year")

In [14]:
df["year"] = df["year"].astype(int)

In [15]:
df = df.loc[df["year"].isin(time_span)]

In [16]:
df["inflation_rate"] = df["inflation_rate"].apply(
    lambda x: float(x.replace(",", ".").strip("%")) / 100.0
)

In [17]:
df.head()

,year,inflation_rate
43,1980,0.212
42,1981,0.178
41,1982,0.165
40,1983,0.147
39,1984,0.108


In [18]:
df.to_csv(clean_data_dir / "inflation_rate.csv", sep="\t", index=False)

### Inflation Target

(approximation reading papers)

The idea of targeting inflation has developed in the early 2000's once the ECB was introduced. 
Therefore, no clear-cut data was found. However, we know that the Italy was aiming at an inflation of about 5-7 % in the early 80's which started decerasing during the course of the years down to the iconic 2 %. 

Down here is the data estimated by Mario Sarcinelli.

In [19]:
years = list(range(1980, 2003))

inflation_target = [12, 11.5, 10.5, 10, 10, 7, 6, 4, 4.5, 4, 4.5, 5, 4.5, 4.5, 3.5, 4.7, 4., 3, 2, 2, 2, 2, 2]

df = pd.DataFrame({'year': years, 'inflation_target': inflation_target})

In [20]:
df.to_csv(clean_data_dir / "inflation_target.csv", sep="\t", index=False)

### Gdp

(Macrotrends)

In [21]:
df = pd.read_csv(
    raw_data_dir / "italy-gdp-gross-domestic-product.csv",
    sep=",",
    usecols=["date", "GDP ( Billions of US $)"],
)

In [22]:
df = df.rename(columns={"GDP ( Billions of US $)": "gdp"})

In [23]:
def date_to_year(date):
    date_object = datetime.strptime(date, "%Y-%m-%d")
    year = date_object.year
    return year

In [24]:
df["year"] = df["date"].apply(lambda x: date_to_year(x))
df.drop("date", axis=1, inplace=True)

In [25]:
df = df[["year", "gdp"]]

In [26]:
df = df.loc[df["year"].isin(time_span)]

In [27]:
df.head()

,year,gdp
20,1980,477.256776
21,1981,430.702851
22,1982,427.272646
23,1983,443.042374
24,1984,437.887689


In [28]:
df.to_csv(clean_data_dir / "gdp.csv", sep="\t", index=False)

### Output Gap 

(OECD, Nasdaq)

In [29]:
# work on oecd
df_oecd = pd.read_csv(
    raw_data_dir / "EO108_INTERNET_27032023112456648.csv",
    sep=",",
    usecols=["Value", "Time"]
)

# fill gaps with nasdaq
df_nasdaq = pd.read_csv(
    raw_data_dir / "nasdaq_output_gap.csv",
    sep=",",
)

In [30]:
df_oecd = df_oecd.rename(columns={"Time": "year", "Value": "output_gap"})

In [31]:
df_oecd.head()

,year,output_gap
0,1985,-2.239437
1,1986,-1.493276
2,1987,-0.898263
3,1988,0.883434
4,1989,1.800495


In [32]:
df_oecd["year"].unique()
# need from 1980 to 1984

array([1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995,
       1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006,
       2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017,
       2018, 2019, 2020, 2021, 2022])

In [33]:
df_oecd = df.loc[df_oecd["year"].isin(time_span)]

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [ ]:
df_nasdaq = df_nasdaq.rename(columns={"Value": "output_gap"})

In [ ]:
df_nasdaq["year"] = df_nasdaq["Date"].apply(lambda x: date_to_year(x))
df_nasdaq.drop("Date", axis=1, inplace=True)

In [ ]:
df_nasdaq = df_nasdaq[["year", "output_gap"]]

In [ ]:
df_nasdaq = df_nasdaq.sort_values("year")

In [ ]:
df_nasdaq = df_nasdaq.loc[df_nasdaq["year"].isin([year for year in range(1980, 1985)])]

In [ ]:
df = pd.concat([df_nasdaq, df_oecd])

In [ ]:
df.to_csv(clean_data_dir / "output_gap.csv", sep="\t", index=False)

# Additional Regressors

### Unemployment

(FRED)

In [ ]:
df = pd.read_csv(
    raw_data_dir / "LRHUTTTTITA156S.csv",
    sep=",",
)

In [ ]:
df = df.rename(columns={"DATE": "year", "LRHUTTTTITA156S": "unemployment_rate"})

In [ ]:
df["year"] = df["year"].apply(lambda x: date_to_year(x))

In [ ]:
df = df.loc[df["year"].isin(time_span)]

In [ ]:
df.head()

In [ ]:
df.to_csv(clean_data_dir / "unemployment_rate.csv", sep="\t", index=False)

### Exchange Rate

In [ ]:
df = pd.read_csv(
    raw_data_dir / "USD_serie_storica_giornaliera .csv",
    sep=",",
    usecols=["Data di Riferimento", "Quotazione"]
)

In [ ]:
df = df.rename(columns={"Data di Riferimento": "year", "Quotazione": "exchange_rate"})

In [ ]:
df = df.sort_values("year")

In [ ]:
df["year"] = df["year"].apply(lambda x: date_to_year(x))

In [ ]:
df = df.drop_duplicates("year", keep='last')

In [ ]:
df = df.loc[df["year"].isin(time_span)]

In [ ]:
add = pd.DataFrame(
[[2002, 1950.000]],
    columns=["year", "exchange_rate"]
)
add

In [ ]:
df = pd.concat([df, add])

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df.to_csv(clean_data_dir / "exchange_rate.csv", sep="\t", index=False)

### Foreign Interest Rate

In [ ]:
df = pd.read_csv(
    raw_data_dir / "US10y_bond.csv",
    sep=",",
)

In [ ]:
df = df.rename(columns={"YEAR": "year", "Value": "US_interest_rate"})

In [ ]:
df = df.loc[df["year"].isin(time_span)]

In [ ]:
df.head()

In [ ]:
df.to_csv(clean_data_dir / "us_bond_yield.csv", sep="\t", index=False)